In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

### 回傳Leaderboards上某頁數的全部召喚師ID(伺服器region =1:KR, =2:NA, =3:EUW, =4:EUNE)

In [2]:
def get_ID(region:int, page:int)->list:
    container = []
    soup = BeautifulSoup(requests.get(f'https://{["www", "na", "euw", "eune"][region]}.op.gg/ranking/ladder/page={str(page)}').text, 'html.parser')      
    if page == 1:
        container = [i.text for i in soup.find_all("a", class_="ranking-highest__name")]
    return container + [i.text for i in soup.find_all("td", class_="select_summoner ranking-table__cell ranking-table__cell--summoner")]

### 回傳某召喚師ID的最近二十場內有效牌位對戰英雄(左上,左野,左中,左下,左輔,右上,右野,右中,右下,右輔,1=左贏0=左輸)

In [3]:
def get_match(region:int, summoner_id:str)->pd.DataFrame:
    soup = BeautifulSoup(requests.get(f'https://{["www", "na", "euw", "eune"][region]}.op.gg/summoner/userName={summoner_id.replace(" ", "+")}').text, 'html.parser')
    df = np.reshape(np.matrix([i.text for i in soup.find_all("div", class_="Image16")]), (-1, 10))
    df = np.append(df, np.reshape(np.matrix([i.get_text(strip=True) for i in soup.select('div.GameSettingInfo > div.ChampionName')]), (-1, 1)), 1)
    df = np.append(df, np.reshape(np.matrix([i.get_text(strip=True) for i in soup.find_all("div", class_="GameType")]), (-1, 1)), 1)
    df = np.append(df, np.reshape(np.matrix([i.get_text(strip=True) for i in soup.find_all("div", class_="GameResult")]), (-1, 1)), 1)
    df = pd.DataFrame(np.squeeze(np.asarray(df)))
    df = df[~df[12].str.contains("Remake") & df[11].str.contains("Ranked")]
    df['result'] = np.where((((df[0]==df[10])|(df[1]==df[10])|(df[2]==df[10])|(df[3]==df[10])|(df[4]==df[10]))&(df[12].str.contains("Victory")))|
                            (((df[5]==df[10])|(df[6]==df[10])|(df[7]==df[10])|(df[8]==df[10])|(df[9]==df[10]))&(df[12].str.contains("Defeat"))), 1, 0)
    df.drop(df.columns[[10, 11, 12]], axis = 1, inplace = True)
    return df

### 回傳第一頁到第n頁的所有召喚師對位勝率和結果，並且過濾掉可能重複的對戰

In [4]:
def get_many_match(region:int, n:int)->pd.DataFrame:
    df = pd.DataFrame()
    for i in range(1, n + 1):
        for j in get_ID(region, i):
            df = df.append(get_match(region, j))
    df.drop_duplicates(keep = 'first', inplace = True)
    df.reset_index(drop=True)
    dict = {'Aatrox': '266', 'Ahri': '103', 'Akali': '84', 'Alistar': '12', 'Amumu': '32', 'Anivia': '34', 'Annie': '1', 'Aphelios': '523',
            'Ashe': '22', 'Aurelion Sol': '136', 'Azir': '268', 'Bard': '432', 'Blitzcrank': '53', 'Brand': '63', 'Braum': '201', 'Caitlyn': '51',
            'Camille': '164', 'Cassiopeia': '69', "Cho'Gath": '31', 'Corki': '42', 'Darius': '122', 'Diana': '131', 'Dr. Mundo': '36', 'Draven': '119',
            'Ekko': '245', 'Elise': '60', 'Evelynn': '28', 'Ezreal': '81', 'Fiddlesticks': '9', 'Fiora': '114', 'Fizz': '105', 'Galio': '3', 
            'Gangplank': '41', 'Garen': '86', 'Gnar': '150', 'Gragas': '79', 'Graves': '104', 'Hecarim': '120', 'Heimerdinger': '74', 'Illaoi': '420',
            'Irelia': '39', 'Ivern': '427', 'Janna': '40', 'Jarvan IV': '59', 'Jax': '24', 'Jayce': '126', 'Jhin': '202', 'Jinx': '222',
            "Kai'Sa": '145', 'Kalista': '429', 'Karma': '43', 'Karthus': '30', 'Kassadin': '38', 'Katarina': '55', 'Kayle': '10', 'Kayn': '141',
            'Kennen': '85', "Kha'Zix": '121', 'Kindred': '203', 'Kled': '240', "Kog'Maw": '96', 'LeBlanc': '7', 'Lee Sin': '64', 'Leona': '89', 
            'Lissandra': '127', 'Lucian': '236', 'Lulu': '117', 'Lux': '99', 'Malphite': '54', 'Malzahar': '90', 'Maokai': '57', 'Master Yi': '11',
            'Miss Fortune': '21', 'Mordekaiser': '82', 'Morgana': '25', 'Nami': '267', 'Nasus': '75', 'Nautilus': '111', 'Neeko': '518', 'Nidalee': '76',
            'Nocturne': '56', 'Nunu & Willump': '20', 'Olaf': '2', 'Orianna': '61', 'Ornn': '516', 'Pantheon': '80', 'Poppy': '78', 'Pyke': '555',
            'Qiyana': '246', 'Quinn': '133', 'Rakan': '497', 'Rammus': '33', "Rek'Sai": '421', 'Renekton': '58', 'Rengar': '107', 'Riven': '92',
            'Rumble': '68', 'Ryze': '13', 'Sejuani': '113', 'Senna': '235', 'Sett': '875', 'Shaco': '35', 'Shen': '98', 'Shyvana': '102',
            'Singed': '27', 'Sion': '14', 'Sivir': '15', 'Skarner':'72', 'Sona': '37', 'Soraka': '16', 'Swain': '50', 'Sylas': '517', 
            'Syndra': '134', 'Tahm Kench': '223', 'Taliyah': '163', 'Talon': '91', 'Taric': '44', 'Teemo': '17', 'Thresh': '412', 'Tristana': '18',
            'Trundle': '48', 'Tryndamere': '23', 'Twisted Fate': '4', 'Twitch': '29', 'Udyr': '77', 'Urgot': '6', 'Varus': '110', 'Vayne': '67',
            'Veigar': '45', "Vel'Koz": '161', 'Vi': '254', 'Viktor': '112', 'Vladimir': '8', 'Volibear': '106', 'Warwick': '19', 'Wukong': '62', 
            'Xayah': '498', 'Xerath': '101', 'Xin Zhao': '5', 'Yasuo': '157', 'Yorick': '83', 'Yuumi': '350', 'Zac': '154', 'Zed': '238',
            'Ziggs': '115', 'Zilean': '26', 'Zoe': '142', 'Zyra': '143'}
    container = []
    for index, row in df.iterrows():
        for i in range(0, 5):
            url = f'https://www.op.gg/champion/ajax/statistics/counterChampion/championId={dict[row[i]]}&targetChampionId={dict[row[i + 5]]}&position={["top", "jungle", "mid", "adc", "support"][i]}'
            list = BeautifulSoup(requests.get(url).text, 'html.parser').find_all("td", class_="champion-matchup-data")
            if list and (i == 0 or i == 2 or i == 3):
                container.append(float(list[10].get_text(strip=True).replace("%", "")) * 0.01)         
                container.append(float(list[0].get_text(strip=True).replace("%", "")) / 
                                 (float(list[0].get_text(strip=True).replace("%", "")) + float(list[1].get_text(strip=True).replace("%", ""))))
                container.append(float(list[2].get_text(strip=True).partition(' :')[0]) / 
                                 (float(list[2].get_text(strip=True).partition(' :')[0]) + float(list[3].get_text(strip=True).partition(' :')[0])))
                container.append(float(list[4].get_text(strip=True).replace("%", "")) / 
                                 (float(list[4].get_text(strip=True).replace("%", "")) + float(list[5].get_text(strip=True).replace("%", ""))))
                container.append(float(list[6].get_text(strip=True).replace(",", "")) / 
                                 (float(list[6].get_text(strip=True).replace(",", "")) + float(list[7].get_text(strip=True).replace(",", ""))))
            elif list and i == 1:
                container.append(float(list[6].get_text(strip=True).replace("%", "")) * 0.01)         
                container.append(float(list[0].get_text(strip=True).partition(' :')[0]) / 
                                 (float(list[0].get_text(strip=True).partition(' :')[0]) + float(list[1].get_text(strip=True).partition(' :')[0])))
                container.append(float(list[2].get_text(strip=True).replace("%", "")) /
                                 (float(list[2].get_text(strip=True).replace("%", "")) + float(list[3].get_text(strip=True).replace("%", ""))))
                container.append(float(list[4].get_text(strip=True).replace(",", "")) /
                                 (float(list[4].get_text(strip=True).replace(",", "")) + float(list[5].get_text(strip=True).replace(",", ""))))
            elif list and i == 4:
                container.append(float(list[6].get_text(strip=True).replace("%", "")) * 0.01)         
                container.append(float(list[0].get_text(strip=True).replace("%", "")) / 
                                 (float(list[0].get_text(strip=True).replace("%", "")) + float(list[1].get_text(strip=True).replace("%", ""))))
                container.append(float(list[2].get_text(strip=True).partition(' :')[0]) /
                                 (float(list[2].get_text(strip=True).partition(' :')[0]) + float(list[3].get_text(strip=True).partition(' :')[0])))
                container.append(float(list[4].get_text(strip=True).replace("%", "")) / 
                                 (float(list[4].get_text(strip=True).replace("%", "")) + float(list[5].get_text(strip=True).replace("%", ""))))
            elif not list and (i == 0 or i == 2 or i == 3):
                container += [0.5, 0.5, 0.5, 0.5, 0.5]
            else:
                container += [0.5, 0.5, 0.5, 0.5]
    df = np.append(df, np.reshape(np.matrix(container), (-1, 23)), 1)    
    df = pd.DataFrame(df)
    df.drop(df.columns[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], axis = 1, inplace = True)
    df.columns = ['result', 'top_win_rate', 'top_lane_kill_rate', 'top_kda', 'top_kill_participate', 'top_damage',
                  'jg_win_rate', 'jg_kda', 'jg_kill_participate', 'jg_damage', 
                  'mid_win_rate', 'mid_lane_kill_rate', 'mid_kda', 'mid_kill_participate', 'mid_damage', 
                  'adc_win_rate', 'adc_lane_kill_rate', 'adc_kda', 'adc_kill_participate', 'adc_damage',
                  'sup_win_rate', 'sup_lane_kill_rate', 'sup_kda', 'sup_kill_participate']
    return df

### 實際爬爬看

In [5]:
get_many_match(2,1)

In [ ]:
# to_csv("test.csv")